In [1]:
#define hostname
HOSTNAME = '192.168.1.100'

In [3]:
import pyrpl
p = pyrpl.Pyrpl(config="",  # do not use a config file 
                hostname=HOSTNAME)
rp = p.redpitaya  # shortcut for the the redpitaya handler

INFO:pyrpl.redpitaya:Successfully connected to Redpitaya with hostname 192.168.178.26.


In [26]:
# setup both asgs
for asg in [rp.asg0, rp.asg1]:
    asg.setup(waveform='square', 
              amplitude=0.5, 
              offset=0, 
              frequency=1e6, 
              output_direct='off',
              trigger_source='immediately',
              )

# make the assertion that signals are not synhronized
ch1, ch2 = rp.scope.curve()
assert ((ch1-ch2)!=0).any(), 'asg channel outputs are identical'

# manually check on the scope that signals are not synchronized
rp.scope.setup(input1='asg0',
               input2='asg1',
               duration=1e-6,
               average=False,
               trigger_source='asg0',
               rolling_mode=False,
               running_state='running_continuous'
               )

In [27]:
# setup the trigger pin
rp.hk.expansion_P0_output = True
rp.hk.expansion_P0 = False

# setup asg trigger
for asg in [rp.asg0, rp.asg1]:
    asg.trigger_source = "ext_positive_edge"
    
# launch the trigger by creating a 0-to-1 transition on trigger pin
rp.hk.expansion_P0 = True
rp.hk.expansion_P0 = False

# make the assertion that signals are synhronized
ch1, ch2 = rp.scope.curve()
assert ((ch1-ch2)==0).all(), 'asg channel outputs are not identical'

# now check that the asg signals are synchronized on the scope...
rp.scope.setup(running_state='running_continuous')